In [33]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8m-seg-custom.pt")

img_real = cv2.imread("E:\\AIML\\ZIP_TAG_PROJECT\\attempt_06\\final_check_imgs\\Without_Head.jpg")

prediction = model.predict(source = img_real, show = True, save = True, hide_labels = False, hide_conf = False, conf = 0.5, save_txt = False, save_crop = False, line_thickness = 2)


WARNING  Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.0.49  Python-3.10.9 torch-1.13.1+cpu CPU
YOLOv8m-seg summary (fused): 245 layers, 27222963 parameters, 0 gradients, 110.0 GFLOPs

0: 640x480 (no detections), 733.8ms
Speed: 1.0ms preprocess, 733.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict8


In [34]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import imutils

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

# img_real = cv2.imread("E:\AIML\ZIP_TAG_PROJECT\attempt_06\final_check_imgs\1.png")
img = cv2.resize(img_real, (700, 700))

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (15, 15), 0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel,iterations=3)

result_img = closing.copy()
contours,hierachy = cv2.findContours(result_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
hitung_objek = 0

pixelsPerMetric = None

for cnt in contours:
    area = cv2.contourArea(cnt)

    if area < 1000 or area > 120000:
        continue

    orig = img.copy()
    box = cv2.minAreaRect(cnt)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 64), 2)

    for (x, y) in box:
        cv2.circle(orig, (int(x), int(y)), 5, (0, 255, 64), -1)
   
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)

    cv2.circle(orig, (int(tltrX), int(tltrY)), 0, (0, 255, 64), 5)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 0, (0, 255, 64), 5)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 0, (0, 255, 64), 5)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 0, (0, 255, 64), 5)

    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
            (255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
            (255, 0, 255), 2)
    
    lebar_pixel = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
    panjang_pixel = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

    if pixelsPerMetric is None:
        pixelsPerMetric = lebar_pixel
        pixelsPerMetric = panjang_pixel
    lebar = lebar_pixel
    panjang = panjang_pixel

    print("L : ", format(lebar_pixel/25.5))
    print("P : ", format(panjang_pixel/25.5))

    cv2.putText(orig, "L: {:.1f}CM".format(lebar_pixel/25.5),(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0,0,255), 2)
    cv2.putText(orig, "P: {:.1f}CM".format(panjang_pixel/25.5),(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0,0,255), 2)
    #cv2.putText(orig,str(area),(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 0.6,(0,0,0),2)
    hitung_objek+=1


cv2.putText(orig, "Image: {}".format(hitung_objek),(10,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2, cv2.LINE_AA)  
# cv2.imshow('Kamera',orig)
cv2.imwrite("E:\\AIML\\ZIP_TAG_PROJECT\\attempt_06\\dimensions_output_folder\\output.jpg", orig)
# cv2.waitKey(0)

L :  0.5504183861811058
P :  14.525111162631864


True

In [39]:

# Initilising the result variable to ZERO :     1 => GOOD    0 => BAD
HEAD = 1

# First Check using head condition
if len(prediction[0].boxes) == 1:
    print("Head is there")
    HEAD = 1
else:
    print("Head is not there")
    HEAD = 0

# Assigning values of dimensions of the body to VARIABLES and creating an ARRAY.
L = format(lebar_pixel/25.5)
R = format(panjang_pixel/25.5)
arr1 = [L, R]

# print(arr1)
# print(type(arr1))

TAIL = 1
# Checking the tail condition
for i in range(len(arr1)):
    val_of_arr1_in_int_01 = float(arr1[i])
    val_of_arr1_in_int_02 = float(arr1[i+1])

    if (val_of_arr1_in_int_01 > 12.0 or val_of_arr1_in_int_02 > 12.0) :
        print("Tail Present ")
        TAIL = 1
    else :
        print("TAIL CUT")
        TAIL = 0
    break


# THE LAST RESULT CONDITION :
if HEAD == 1 and TAIL == 1:
    print("GOOD PRODUCT ... :)")
else :
    print("BAD PRODUCT ... :(")

Head is not there
Tail Present 
BAD PRODUCT ... :(


: 